# Data cleaning

## Clarifications
* ```Isolierbereich``` signifies if inhabitant was in the isolation station or not?
* ```Checkpoint```?
* difference between ```Besucherzone/Garten``` and ```Garten_Aufenthalt```?
* ```WB_übergreifender_BW_Kontakt_```? 
* employees no longer have infection information
* ```Tischnummer=0``` should be NaN?

## Contact defining columns
Direct (strong)
* ```Zimmernummer```
* ```vis-a-vis_Tischnachbar```
* ```Tischnummer```
* ```Isolierbereich```
* ```Kontakt_interne_MA``` && ```MA_ohne_Maske```

Indirect (medium)
* ```Wohnzimmer Aufenthalt```
* ```Aula_Aufenthalt```
* ```Kaffeehaus_ohne_Plexiglas_Aufenthalt```
* ```Plexiglas_vis-a-vis_Tischnachbar```
* ```Hl_Messe_Aufenthalt```
* ```Speisesaal```
* ```Kontakt_interne_MA``` && (```MA_mit_MNS``` | ```MA_mit_FFP2_ohne_Ventil``` | ```MA_mit_FFP2_mit_Ventil```)

Indirect (weak)
* ```Garten_Aufenthalt```
* ```Kaffeehaus_mit_Plexiglas_Aufenthalt```
* ```Wohnbereich```

External contacts
* ```Arzt_Ordination_Kontakt```
* ```KH_ambulant```
* ```KH_stationär```
* ```Anzahl_Besucher``` $\rightarrow$ linear scaling of risk with number of visitors
* ```externe_Aktivität```
* ```Kontakt_externe_MA``` && (```MA_mit_MNS``` | ```MA_mit_FFP2_ohne_Ventil``` | ```MA_mit_FFP2_mit_Ventil```)
* ```Kontakt_externe_MA``` && ```MA_ohne_Maske```

Mobility $\rightarrow$ treat as levels [0, 1, 2, 3]
* ```mobil_hypermobil```
* ```mobil_ohne_Einschräkung```
* ```mobil_Rollstuhl```
* ```mobil_inmobil```

## Analysis ideas
Do the following for every ```Haus```

1. measure link strengths from data:
    * identify index cases
    * for every week: logistic regression person_infected ~ shared_room + shared_table + ...
2. construct contact network between all inhabitants and employees:
    * there is a contact/link if at least one of the direct or indirect contact possibilities are shared
    * at first: uniform link strength

Kontakte:
* gemeinsamer Wohnbereich
* gemeinsamer Tisch
* gemeinsames Zimmer

* S(E)IR Modell
* Poissonprozesse mit Konstanter Rate
* Andere Parameter in Regression screenen
* Freie parameter: 
    - Hintergrundwahrscheinlichkeit / Ansteckung von Extern
    - effektive Infektiösitätsdauer

In [183]:
import pandas as pd
import numpy as np

In [184]:
def ParseDate(date, week_type='start'):
    '''
    date strings have the format DD.MM. - DD.MM., where the first DD.MM. is the
    start of the time period (week) and the second DD.MM. is the end of the time
    period
    '''
    if week_type == 'start':
        date = date.split('_')[0]
    else:
        date = date.split('_')[1]
        
    month = int(date.split('.')[1])
    day = int(date.split('.')[0])
    return pd.to_datetime('2020-{}-{}'.format(month, day))

In [185]:
'''
# zip PW: ca612b280809422fa1b8ff689cdf97b8
import uuid
for WB in [2, 4]:
    f = pd.ExcelFile('../../data/nursing_homes/WB{}.xlsx'.format(WB))
    names = f.sheet_names 
    name_table = pd.DataFrame()
    name_table['name'] = names
    name_table['ID'] = [uuid.uuid4().hex for i in range(len(name_table))]
    name_table.to_csv('../../data/nursing_homes/name_table_WB{}.csv'.format(WB),
                      index=False)
'''

"\n# zip PW: ca612b280809422fa1b8ff689cdf97b8\nimport uuid\nfor WB in [2, 4]:\n    f = pd.ExcelFile('../../data/nursing_homes/WB{}.xlsx'.format(WB))\n    names = f.sheet_names \n    name_table = pd.DataFrame()\n    name_table['name'] = names\n    name_table['ID'] = [uuid.uuid4().hex for i in range(len(name_table))]\n    name_table.to_csv('../../data/nursing_homes/name_table_WB{}.csv'.format(WB),\n                      index=False)\n"

In [111]:
# parse sheets into a single data file
for WB in [2, 4]:
    print('WB: {}'.format(WB))
    data = pd.DataFrame()
    name_table = pd.read_csv('../../data/nursing_homes/name_table_WB{}.csv'\
                             .format(WB))
    for i, row in name_table.iterrows():
        if row['name'] != 'Eckdaten':
            #print(i)
            tmp = pd.read_excel('../../data/nursing_homes/WB{}.xlsx'.format(WB),
                          sheet_name=row['name'])
            if '16.03.2020 - 26.07.2020 ' in tmp.columns:
                print(row['name'])
            tmp['ID'] = row['ID']
            data = pd.concat([data, tmp])
        
    data.to_csv('../../data/nursing_homes/WB{}_combined_raw.csv'.format(WB),
               index=False)

WB: 2
WB: 4


In [188]:
for WB in [2, 4]:
    data = pd.read_csv('../../data/nursing_homes/WB{}_combined_raw.csv'.format(WB))
    data = data.rename(columns={'Kontakt_ interne_MA':'Kontakt_interne_MA',
                                'WB_übergreifender_BW_Kontakt_':'WB_übergreifender_BW_Kontakt',
                                'MA_ohne Maske':'MA_ohne_Maske',
                                'Zimmer-Nr.':'Zimmernummer',
                                'MA_mit_MNS ':'MA_mit_MNS',
                                'mobil_ohne_Einschränkung':'mobil_ohne_Einschränkung'})
    data = data.dropna(subset=['Datum'])
    data['week'] = data['Datum'].apply(ParseDate)
    data = data.drop(columns=['Datum'])
    # drops second column for "WB_übergreifender_BW_Kontakt", which contains next
    # to no data (6 "0" entries)
    data = data.loc[:,~data.columns.duplicated()]

    data['Geschlecht'] = data['Geschlecht'].replace({' w':'w'})
    data['Zimmernummer'] = data['Zimmernummer'].replace({'0':np.nan, '999':np.nan})
    data['Tischnummer'] = data['Tischnummer'].replace({0:np.nan})

    for c in ['Wohnbereich', 'Isolierbereich', 'Alter', 'Verstorben', 
     'mobil_hypermobil', 'mobil_ohne_Einschränkung', 'mobil_Rollstuhl',
     'mobil_inmobil', 'Pflegestufe', 'Covid19_pos',
     'Wohnzimmer_Aufenthalt', 'Speisesaal', 'Tischnummer', 'vis-a-vis_Tischnachbar',
     'Plexiglas_vis-a-vis_Tischnachbar', 'Kontakt_externe_MA', 'Kontakt_interne_MA',
     'Arzt_Ordination_Kontakt', 'KH_ambulant', 'KH_stationär', 'Checkpoint',
     'Anzahl_Besucher', 'Hl_Messe_Aufenthalt', 'Aula_Aufenthalt', 'externe_Aktivität',
     'Garten_Aufenthalt', 'Kaffeehaus_ohne_Plexiglas_Aufenthalt',
     'Kaffeehaus_mit_Plexiglas_Aufenthalt', 'MA_ohne_Maske', 'MA_mit_MNS',
     'MA_mit_FFP2', 'PCR_Test', 'Besucherzone/Garten', 'WB_übergreifender_BW_Kontakt']:
        data[c] = data[c].replace({999:np.nan})
        
    # remove/replace symbols from column-names that confuse the patsy
    # formula api from statsmodels
    for c in data.columns:
        new_c = c.replace('.','')
        new_c = new_c.replace('(','')
        new_c = new_c.replace(')','')
        new_c = new_c.replace(' ','_')
        new_c = new_c.replace('+','_plus_')
        new_c = new_c.replace('-','_')
        new_c = new_c.replace('/','_und_')
        new_c = new_c.replace(',','')
        new_c = new_c.replace('ü','ue')
        new_c = new_c.replace('ö','oe')
        new_c = new_c.replace('ä','ae')
        data = data.rename(columns={c:new_c})
        
    data.to_csv('../../data/nursing_homes/WB{}_combined_clean.csv'.format(WB), index=False)